In [ ]:
import sys
sys.path.append('..')
import neural_policies
import main
import game_utils
import yaml
import torch
import policy_gradient
import numpy as np
import pyspiel

seed = 1
torch.random.manual_seed(seed)
np.random.seed(seed)

In [ ]:
# game = game_utils.create_random_normal_game(5)
game = pyspiel.load_game("leduc_poker")
nn_config_file = "../config/mlp.yaml"
reinforce_config_file = "../config/reinforce_train.yaml"
nn_config = yaml.safe_load(open(nn_config_file, "r"))
reinforce_config = yaml.safe_load(open(reinforce_config_file, "r"))

In [ ]:
nash_policy, nash_value = main.nash_equilibrium_policy_and_value(game)
print("nash value", nash_value)
nn_player = 0
nn_policy = neural_policies.create_policy_net(game, nn_config)
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)

In [ ]:
hypernet = main.train_hypernet_actionoutput(game, nn_config, nn_player, nn_policy, "../config/hypernet.yaml", "../config/hypernet_reinforce_train.yaml")

In [ ]:
nn_policy = neural_policies.create_policy_net(game, nn_config)
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)
main.tabular_br(game, nn_player, nn_tab_policy)
main.policy_gradient_br(
    game, nn_config, reinforce_config, nn_player, nn_policy, nn_tab_policy
)
main.hypernet_br(game, hypernet, nn_player, nn_policy, nn_tab_policy)

lst_policy = None

In [ ]:
# print(game.row_utilities())

new_input_nn = neural_policies.create_policy_net(game, nn_config)
input_probs = neural_policies.nn_to_tabular_policy(game, new_input_nn, nn_player).action_probability_array
# input_probs = neural_policies.get_nn_probs(new_input_nn, game.new_initial_state(), nn_player)
# print("input probs", input_probs)

playable_states = game_utils.all_game_states(game, 1-nn_player)
state = playable_states[0]

# print(neural_policies.get_hypernet_probs(hypernet, new_input_nn, state, 1-nn_player))
# print(hypernet(new_input_nn, torch.tensor(state.information_state_tensor(1-nn_player))))
nn_response = neural_policies.nn_to_tabular_policy(game, hypernet, 1-nn_player, new_input_nn)
# print("nn response", nn_response.action_probability_array)
br = main.compute_best_response_tabular_policy(game, nn_tab_policy, 1 - nn_player)
print(lst_policy)
if lst_policy is not None:
    cur_policy = br.action_probability_array
    # find rows that differ
    diff = np.where(cur_policy != lst_policy)
    print("diff", diff)
lst_policy = br.action_probability_array
print(br.action_probability_array)

# neural_br_policy = neural_policies.create_policy_net(game, nn_config)
# trainer = policy_gradient.PolicyGradientTrainer(
#     game, neural_br_policy, train_params=reinforce_config
# )
# trainer.train_best_response(nn_policy, 1-nn_player)
# print(neural_br_policy(torch.tensor(game.new_initial_state().information_state_tensor(1-nn_player))))
# print(neural_policies.get_nn_probs(neural_br_policy, game.new_initial_state(), 1-nn_player))

In [ ]:
from pprint import pprint
# pprint(list(hypernet._model.get_weights()))
mm = hypernet._model
[l.bias for l in mm.layers]
# mm.layers[-1].weight